In [1]:
from pytplot import get_data, del_data
from pyspedas import load_data, time_string

import pandas as pd
import numpy as np

In [2]:
#######################################
# データ取得元 & データ作成先定義
#######################################
root_dir = '/Volumes/data/swfpy/kyoto/'
dst_dir  = root_dir + 'DST/'


# output_dir = '/Volumes/data/swfpy/ml/storm_datasets/basic/DST/'
output_dir = '/Volumes/data/swfpy/ml/storm_datasets/Gruet_2018/DST/'

# event定義ファイル
events_file='/home/hirotoshi/PycharmProjects/swfpy/playground/worklogs/magnetic_storm_events.csv'


In [3]:
events_df = pd.read_csv(events_file)


In [6]:
# events_dfのData_start, Data_endに含まれるデータを準備
from glob import glob
from datetime import datetime, timedelta

def make_dst_event_df(start_dt, end_dt, dst_dir):
    def get_DST_file(dt, dst_root_dir):
        tgt_dir = dst_root_dir + str(dt.year) + '/'
        glob_pattern = tgt_dir + 'DST_' + str(dt.year)+ str(dt.month).rjust(2, '0') + '*'
        return glob(glob_pattern)[0]
    
    # 必要なファイルを開けてつなげる (ファイルが3つになる場合はまず存在しないと考える。磁気嵐1ヶ月以上)
    file_s = get_DST_file(start_dt, dst_dir)
    file_e = get_DST_file(end_dt, dst_dir)

    dst_df = pd.read_csv(file_s)
    if file_s != file_e:
        dst_df_end = pd.read_csv(file_e)
        dst_df = pd.concat([dst_df, dst_df_end])

    # Time をdatetime式に変更
    # start と endに含まれる 部分だけ抜き出し
    start_int = int(start_dt.strftime('%Y%m%d%H'))
    end_int   = int(end_dt.strftime('%Y%m%d%H'))

    dst_event_df = dst_df[(dst_df['Time'] >= start_int) & (dst_df['Time'] <= end_int)]
    dst_event_df = dst_event_df.reset_index(drop=True)
    
    def ut24to0nextday(ymdh_int):
        ymd  = str(ymdh_int)[0:8]
        hour = str(ymdh_int)[8:10]

        if hour == '24':
            dt_ymd = datetime.strptime(ymd, '%Y%m%d')
            dt_ymd = dt_ymd + timedelta(1)
            return int(dt_ymd.strftime('%Y%m%d%H'))
        else:
            return ymdh_int
    
    dst_event_df['Time'] = dst_event_df['Time'].apply(ut24to0nextday)
    
    return dst_event_df


def output2csv_event_df(event_df, start_dt, end_dt, output_dir='./', data_label=''):
    output_path_base = output_dir + 'storm_'
    if data_label != '':
        output_path_base += data_label + '_'
    output_path = output_path_base + start_dt.strftime('%Y%m%d%H') + '_' + end_dt.strftime('%Y%m%d%H') + '.csv'
    event_df.to_csv(output_path, index=False)


In [7]:
# DST
event_set_df = pd.read_csv(events_file)

for intdex, event in event_set_df.iterrows():
    Data_start = event.Data_start
    Data_end   = event.Data_end

    start_dt = datetime.strptime(Data_start, '%Y-%m-%d %H:%M')
    end_dt   = datetime.strptime(Data_end, '%Y-%m-%d %H:%M')    

    dst_event_df = make_dst_event_df(start_dt, end_dt, dst_dir)

    # ファイルを出力
    output2csv_event_df(dst_event_df, start_dt, end_dt, output_dir, 'DST')




In [150]:
## OMNIデータ

del_data()
load_data('omni', ['2001-03-19 04:00', '2001-03-22 21:00'], '', '', '5min')

1. File was downloaded. Location: /Volumes/data/pyspedas/omni/hro_5min/2001/omni_hro_5min_20010301_v01.cdf
Downloaded 1 files.


/home/hirotoshi/PycharmProjects/swfpy/venv/lib/python3.7/site-packages/cdflib/epochs.py:1465: ResourceWarning: unclosed <ssl.SSLSocket [closed] fd=81, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6>
  component = []
/home/hirotoshi/PycharmProjects/swfpy/venv/lib/python3.7/site-packages/cdflib/epochs.py:1465: ResourceWarning: unclosed <ssl.SSLSocket [closed] fd=82, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6>
  component = []
/home/hirotoshi/PycharmProjects/swfpy/venv/lib/python3.7/site-packages/cdflib/epochs.py:1465: ResourceWarning: unclosed <ssl.SSLSocket [closed] fd=83, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6>
  component = []


Loaded 44 variables.
Time clip was applied to: IMF
Time clip was applied to: PLS
Time clip was applied to: IMF_PTS
Time clip was applied to: PLS_PTS
Time clip was applied to: percent_interp
Time clip was applied to: Timeshift
Time clip was applied to: RMS_Timeshift
Time clip was applied to: Time_btwn_obs
Time clip was applied to: F
Time clip was applied to: BX_GSE
Time clip was applied to: BY_GSE
Time clip was applied to: BZ_GSE
Time clip was applied to: BY_GSM
Time clip was applied to: BZ_GSM
Time clip was applied to: RMS_SD_B
Time clip was applied to: RMS_SD_fld_vec
Time clip was applied to: flow_speed
Time clip was applied to: Vx
Time clip was applied to: Vy
Time clip was applied to: Vz
Time clip was applied to: proton_density
Time clip was applied to: T
Time clip was applied to: Pressure
Time clip was applied to: E
Time clip was applied to: Beta
Time clip was applied to: Mach_num
Time clip was applied to: Mgs_mach_num
Time clip was applied to: x
Time clip was applied to: y
Time cli

In [10]:
# targets = ['Vx', 'BZ_GSM', 'BY_GSM', 'proton_density', 'T']
targets = ['flow_speed', 'BZ_GSM', 'F', 'proton_density']
convert_time = np.frompyfunc(lambda x: time_string(x, '%Y%m%d%H%M'), 1, 1)

# omni_out_dir = '/Volumes/data/swfpy/ml/storm_datasets/basic/omni/'
omni_out_dir = '/Volumes/data/swfpy/ml/storm_datasets/Gruet_2018/omni_5min/'

event_set_df = pd.read_csv(events_file)
for intdex, event in event_set_df.iterrows():
    Data_start = event.Data_start
    Data_end   = event.Data_end

    start_dt = datetime.strptime(Data_start, '%Y-%m-%d %H:%M')
    end_dt   = datetime.strptime(Data_end, '%Y-%m-%d %H:%M')    

    del_data()
    load_data('omni', [Data_start, Data_end], '', '', '5min')
    
    omni_event_df = pd.DataFrame()

    for target in targets:
        time = get_data(target)[0]
        pq   = get_data(target)[1]

        omni_event_df['Time'] = convert_time(time)
        omni_event_df[target] = pq
    #     print(nf(time), pq_s)
    output2csv_event_df(omni_event_df, start_dt, end_dt, omni_out_dir, 'OMNI')


1. File was downloaded. Location: /Volumes/data/pyspedas/omni/hro_5min/2000/omni_hro_5min_20000201_v01.cdf
Downloaded 1 files.


18- 7月-19 21:04:56: /home/hirotoshi/PycharmProjects/swfpy/venv/lib/python3.7/site-packages/cdflib/epochs.py:1465: ResourceWarning: unclosed <ssl.SSLSocket [closed] fd=82, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6>
  component = []

18- 7月-19 21:04:56: /home/hirotoshi/PycharmProjects/swfpy/venv/lib/python3.7/site-packages/cdflib/epochs.py:1465: ResourceWarning: unclosed <ssl.SSLSocket [closed] fd=83, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6>
  component = []

18- 7月-19 21:04:56: /home/hirotoshi/PycharmProjects/swfpy/venv/lib/python3.7/site-packages/cdflib/epochs.py:1465: ResourceWarning: unclosed <ssl.SSLSocket [closed] fd=84, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6>
  component = []



Loaded 44 variables.
Time clip was applied to: IMF
Time clip was applied to: PLS
Time clip was applied to: IMF_PTS
Time clip was applied to: PLS_PTS
Time clip was applied to: percent_interp
Time clip was applied to: Timeshift
Time clip was applied to: RMS_Timeshift
Time clip was applied to: Time_btwn_obs
Time clip was applied to: F
Time clip was applied to: BX_GSE
Time clip was applied to: BY_GSE
Time clip was applied to: BZ_GSE
Time clip was applied to: BY_GSM
Time clip was applied to: BZ_GSM
Time clip was applied to: RMS_SD_B
Time clip was applied to: RMS_SD_fld_vec
Time clip was applied to: flow_speed
Time clip was applied to: Vx
Time clip was applied to: Vy
Time clip was applied to: Vz
Time clip was applied to: proton_density
Time clip was applied to: T
Time clip was applied to: Pressure
Time clip was applied to: E
Time clip was applied to: Beta
Time clip was applied to: Mach_num
Time clip was applied to: Mgs_mach_num
Time clip was applied to: x
Time clip was applied to: y
Time cli

18- 7月-19 21:05:13: /home/hirotoshi/PycharmProjects/swfpy/venv/lib/python3.7/site-packages/cdflib/epochs.py:1465: ResourceWarning: unclosed <ssl.SSLSocket [closed] fd=82, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6>
  component = []

18- 7月-19 21:05:13: /home/hirotoshi/PycharmProjects/swfpy/venv/lib/python3.7/site-packages/cdflib/epochs.py:1465: ResourceWarning: unclosed <ssl.SSLSocket [closed] fd=83, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6>
  component = []

18- 7月-19 21:05:13: /home/hirotoshi/PycharmProjects/swfpy/venv/lib/python3.7/site-packages/cdflib/epochs.py:1465: ResourceWarning: unclosed <ssl.SSLSocket [closed] fd=84, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6>
  component = []



Loaded 44 variables.
Time clip was applied to: IMF
Time clip was applied to: PLS
Time clip was applied to: IMF_PTS
Time clip was applied to: PLS_PTS
Time clip was applied to: percent_interp
Time clip was applied to: Timeshift
Time clip was applied to: RMS_Timeshift
Time clip was applied to: Time_btwn_obs
Time clip was applied to: F
Time clip was applied to: BX_GSE
Time clip was applied to: BY_GSE
Time clip was applied to: BZ_GSE
Time clip was applied to: BY_GSM
Time clip was applied to: BZ_GSM
Time clip was applied to: RMS_SD_B
Time clip was applied to: RMS_SD_fld_vec
Time clip was applied to: flow_speed
Time clip was applied to: Vx
Time clip was applied to: Vy
Time clip was applied to: Vz
Time clip was applied to: proton_density
Time clip was applied to: T
Time clip was applied to: Pressure
Time clip was applied to: E
Time clip was applied to: Beta
Time clip was applied to: Mach_num
Time clip was applied to: Mgs_mach_num
Time clip was applied to: x
Time clip was applied to: y
Time cli

...

Loaded 44 variables.
Time clip was applied to: IMF
Time clip was applied to: PLS
Time clip was applied to: IMF_PTS
Time clip was applied to: PLS_PTS
Time clip was applied to: percent_interp
Time clip was applied to: Timeshift
Time clip was applied to: RMS_Timeshift
Time clip was applied to: Time_btwn_obs
Time clip was applied to: F
Time clip was applied to: BX_GSE
Time clip was applied to: BY_GSE
Time clip was applied to: BZ_GSE
Time clip was applied to: BY_GSM
Time clip was applied to: BZ_GSM
Time clip was applied to: RMS_SD_B
Time clip was applied to: RMS_SD_fld_vec
Time clip was applied to: flow_speed
Time clip was applied to: Vx
Time clip was applied to: Vy
Time clip was applied to: Vz
Time clip was applied to: proton_density
Time clip was applied to: T
Time clip was applied to: Pressure
Time clip was applied to: E
Time clip was applied to: Beta
Time clip was applied to: Mach_num
Time clip was applied to: Mgs_mach_num
Time clip was applied to: x
Time clip was applied to: y
Time cli

18- 7月-19 21:25:19: /home/hirotoshi/PycharmProjects/swfpy/venv/lib/python3.7/site-packages/cdflib/epochs.py:1465: ResourceWarning: unclosed <ssl.SSLSocket [closed] fd=82, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6>
  component = []

18- 7月-19 21:25:19: /home/hirotoshi/PycharmProjects/swfpy/venv/lib/python3.7/site-packages/cdflib/epochs.py:1465: ResourceWarning: unclosed <ssl.SSLSocket [closed] fd=83, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6>
  component = []

18- 7月-19 21:25:19: /home/hirotoshi/PycharmProjects/swfpy/venv/lib/python3.7/site-packages/cdflib/epochs.py:1465: ResourceWarning: unclosed <ssl.SSLSocket [closed] fd=84, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6>
  component = []



Loaded 44 variables.
Time clip was applied to: IMF
Time clip was applied to: PLS
Time clip was applied to: IMF_PTS
Time clip was applied to: PLS_PTS
Time clip was applied to: percent_interp
Time clip was applied to: Timeshift
Time clip was applied to: RMS_Timeshift
Time clip was applied to: Time_btwn_obs
Time clip was applied to: F
Time clip was applied to: BX_GSE
Time clip was applied to: BY_GSE
Time clip was applied to: BZ_GSE
Time clip was applied to: BY_GSM
Time clip was applied to: BZ_GSM
Time clip was applied to: RMS_SD_B
Time clip was applied to: RMS_SD_fld_vec
Time clip was applied to: flow_speed
Time clip was applied to: Vx
Time clip was applied to: Vy
Time clip was applied to: Vz
Time clip was applied to: proton_density
Time clip was applied to: T
Time clip was applied to: Pressure
Time clip was applied to: E
Time clip was applied to: Beta
Time clip was applied to: Mach_num
Time clip was applied to: Mgs_mach_num
Time clip was applied to: x
Time clip was applied to: y
Time cli

18- 7月-19 21:25:35: /home/hirotoshi/PycharmProjects/swfpy/venv/lib/python3.7/site-packages/cdflib/epochs.py:1465: ResourceWarning: unclosed <ssl.SSLSocket [closed] fd=82, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6>
  component = []

18- 7月-19 21:25:35: /home/hirotoshi/PycharmProjects/swfpy/venv/lib/python3.7/site-packages/cdflib/epochs.py:1465: ResourceWarning: unclosed <ssl.SSLSocket [closed] fd=83, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6>
  component = []

18- 7月-19 21:25:35: /home/hirotoshi/PycharmProjects/swfpy/venv/lib/python3.7/site-packages/cdflib/epochs.py:1465: ResourceWarning: unclosed <ssl.SSLSocket [closed] fd=84, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6>
  component = []



Loaded 44 variables.
Time clip was applied to: IMF
Time clip was applied to: PLS
Time clip was applied to: IMF_PTS
Time clip was applied to: PLS_PTS
Time clip was applied to: percent_interp
Time clip was applied to: Timeshift
Time clip was applied to: RMS_Timeshift
Time clip was applied to: Time_btwn_obs
Time clip was applied to: F
Time clip was applied to: BX_GSE
Time clip was applied to: BY_GSE
Time clip was applied to: BZ_GSE
Time clip was applied to: BY_GSM
Time clip was applied to: BZ_GSM
Time clip was applied to: RMS_SD_B
Time clip was applied to: RMS_SD_fld_vec
Time clip was applied to: flow_speed
Time clip was applied to: Vx
Time clip was applied to: Vy
Time clip was applied to: Vz
Time clip was applied to: proton_density
Time clip was applied to: T
Time clip was applied to: Pressure
Time clip was applied to: E
Time clip was applied to: Beta
Time clip was applied to: Mach_num
Time clip was applied to: Mgs_mach_num
Time clip was applied to: x
Time clip was applied to: y
Time cli